### Transfer learning... 

Transfer learning as an approach that allows to train models using pretrained weights.. VGG16, RESNet-50, Inception are well known models that have been trained on very large datasets and have state of art results.. 

We will use these models weight to train our network to predict dog's breed.. 

In [2]:
import numpy as np
pre_trained_features = np.load('pretrained-model-weights/DogResnet50Data.npz')
train_ResNet50 = pre_trained_features['train']
valid_ResNet50 = pre_trained_features['valid']
test_ResNet50 = pre_trained_features['test']

#### Load data for training and validation ... 

In [4]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('dogImages/train')
valid_files, valid_targets = load_dataset('dogImages/valid')
test_files, test_targets = load_dataset('dogImages/test')

dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

There are 133 total dog categories.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


#### Model architecture using ResNet_pre_trained weights

In [5]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

ResNet_model = Sequential()
ResNet_model.add(GlobalAveragePooling2D(input_shape=train_ResNet50.shape[1:]))
ResNet_model.add(Dense(133, activation='softmax'))

ResNet_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 133)               272517    
Total params: 272,517
Trainable params: 272,517
Non-trainable params: 0
_________________________________________________________________


### Model compilation...

In [7]:
from keras.optimizers import Adam, Adamax

ResNet_model.compile(loss='categorical_crossentropy', optimizer=Adamax(lr=0.002), metrics=['accuracy'])

In [8]:
from keras.callbacks import ModelCheckpoint  
epochs = 30
batch_size = 64
## comment out this line if you are interested to save the checkpoints... 

#checkpointer = ModelCheckpoint(filepath='saved_models/weights.best_adamax.ResNet50.hdf5', 
#                               verbose=1, save_best_only=True)

#ResNet_model.fit(train_ResNet50, train_targets, 
#          validation_data=(valid_ResNet50, valid_targets),
#          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer], verbose=1)



ResNet_model.fit(train_ResNet50, train_targets, 
          validation_data=(valid_ResNet50, valid_targets),
          epochs=epochs, batch_size=batch_size, verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/30
6680/6680 [==============================] - 1s - loss: 2.3979 - acc: 0.4858 - val_loss: 1.2498 - val_acc: 0.7114
Epoch 2/30
6680/6680 [==============================] - 0s - loss: 0.7923 - acc: 0.8377 - val_loss: 0.8952 - val_acc: 0.7701
Epoch 3/30
6680/6680 [==============================] - 0s - loss: 0.5228 - acc: 0.8994 - val_loss: 0.7702 - val_acc: 0.8012
Epoch 4/30
6680/6680 [==============================] - 0s - loss: 0.3850 - acc: 0.9364 - val_loss: 0.6922 - val_acc: 0.8036
Epoch 5/30
6680/6680 [==============================] - 0s - loss: 0.2968 - acc: 0.9584 - val_loss: 0.6472 - val_acc: 0.8084
Epoch 6/30
6680/6680 [==============================] - 1s - loss: 0.2383 - acc: 0.9717 - val_loss: 0.6253 - val_acc: 0.8240
Epoch 7/30
6680/6680 [==============================] - 0s - loss: 0.1938 - acc: 0.9814 - val_loss: 0.6081 - val_acc: 0.8216
Epoch 8/30
6680/6680 [==============================] - 0s - loss: 0.1600 - ac

In [ ]:
ResNet50_predictions = [np.argmax(ResNet_model.predict(np.expand_dims(feature, axis=0))) for feature in test_ResNet50]

# report test accuracy
test_accuracy = 100*np.sum(np.array(ResNet50_predictions)==np.argmax(test_targets, axis=1))/len(ResNet50_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)